In [1]:
import os
import numpy as np
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4041 --packages org.apache.kafka:kafka_2.11:0.9.0.0,org.apache.kafka:kafka-clients:0.9.0.0  pyspark-shell'

In [2]:
from pyspark import SparkContext
sc = SparkContext("local[1]", "KafkaSendStream") 
from kafka import KafkaProducer
import time

In [12]:
import json
from satori.rtm.client import make_client, SubscriptionMode

endpoint = "wss://open-data.api.satori.com"
appkey = "dC6c33Fbb5ECdAC1Ef2aB77dcBfBB0B0"
channel = "cryptocurrency-market-data"

with make_client(endpoint=endpoint, appkey=appkey) as client:
    print('Connected to Satori RTM!')
    messagebox=[]

    class SubscriptionObserver(object):
        def on_subscription_data(self, data):
            for message in data['messages']:
                messagebox.append(message)

    subscription_observer = SubscriptionObserver()
    client.subscribe(
        channel,
        SubscriptionMode.SIMPLE,
        subscription_observer,
        {'filter':'select * from `cryptocurrency-market-data` where exchange = "Bitstamp" and basecurrency = "USD" and cryptocurrency="BTC"'})
        #  exchange != "ShapheShift" OU type != "ratequote" si on laisse ShapeShift

    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    i=0
    n=10   ## number of inputs
    beta=np.zeros(n)
    beta+=0.1
    beta.shape=(n,1)
    while True:
        x=np.empty(0)
        while len(x)<10:
            if messagebox:
                message=messagebox.pop()
                message=json.dumps(message, ensure_ascii=False)
                message = json.loads(message)
                proxy=(1/(2*np.log(4)))*(np.log(float(message['high'])/float(message['low']))**2)
                x=np.append(x,proxy)
                print(x, len(x))
            time.sleep(1)
        y=float(x.dot(beta))+0.1*np.random.rand(1)[0] # slide 31: y =x^T beta +w / np.random.rand(1)[0] gives number 0<n<1
        message=np.array2string(np.append([i,y],x),separator=",") 
        producer.send('test2', message)
        i=i+1
        print(message)

Connected to Satori RTM!
(array([ 0.00166031]), 1)
(array([ 0.00166031,  0.00176547]), 2)
(array([ 0.00166031,  0.00176547,  0.00176547]), 3)
(array([ 0.00166031,  0.00176547,  0.00176547,  0.00176547]), 4)
(array([ 0.00166031,  0.00176547,  0.00176547,  0.00176547,  0.00176547]), 5)
(array([ 0.00166031,  0.00176547,  0.00176547,  0.00176547,  0.00176547,
        0.00176547]), 6)
(array([ 0.00166031,  0.00176547,  0.00176547,  0.00176547,  0.00176547,
        0.00176547,  0.00176547]), 7)
(array([ 0.00166031,  0.00176547,  0.00176547,  0.00176547,  0.00176547,
        0.00176547,  0.00176547,  0.00176547]), 8)
(array([ 0.00166031,  0.00176547,  0.00176547,  0.00176547,  0.00176547,
        0.00176547,  0.00176547,  0.00176547,  0.00176547]), 9)
(array([ 0.00166031,  0.00176547,  0.00176547,  0.00176547,  0.00176547,
        0.00176547,  0.00176547,  0.00176547,  0.00176547,  0.00176547]), 10)
[ 0.        , 0.06837053, 0.00166031, 0.00176547, 0.00176547, 0.00176547,
  0.00176547, 0.0017

(array([ 0.00197683,  0.00197683,  0.00197683,  0.00197683,  0.00197683,
        0.00197683,  0.00197683,  0.00197683,  0.00197683,  0.00197683]), 10)
[  7.00000000e+00,  8.35842030e-02,  1.97683250e-03,  1.97683250e-03,
   1.97683250e-03,  1.97683250e-03,  1.97683250e-03,  1.97683250e-03,
   1.97683250e-03,  1.97683250e-03,  1.97683250e-03,  1.97683250e-03]
(array([ 0.00197683]), 1)
(array([ 0.00197683,  0.00197683]), 2)
(array([ 0.00197683,  0.00197683,  0.00197683]), 3)
(array([ 0.00197683,  0.00197683,  0.00197683,  0.00197683]), 4)
(array([ 0.00197683,  0.00197683,  0.00197683,  0.00197683,  0.00197683]), 5)
(array([ 0.00197683,  0.00197683,  0.00197683,  0.00197683,  0.00197683,
        0.00197683]), 6)
(array([ 0.00197683,  0.00197683,  0.00197683,  0.00197683,  0.00197683,
        0.00197683,  0.00197683]), 7)
(array([ 0.00197683,  0.00197683,  0.00197683,  0.00197683,  0.00197683,
        0.00197683,  0.00197683,  0.00197683]), 8)
(array([ 0.00197683,  0.00197683,  0.00197683,

(array([ 0.0027811,  0.0027811,  0.0027811,  0.0027811,  0.0027811,
        0.0027811]), 6)
(array([ 0.0027811,  0.0027811,  0.0027811,  0.0027811,  0.0027811,
        0.0027811,  0.0027811]), 7)
(array([ 0.0027811 ,  0.0027811 ,  0.0027811 ,  0.0027811 ,  0.0027811 ,
        0.0027811 ,  0.0027811 ,  0.00279931]), 8)
(array([ 0.0027811 ,  0.0027811 ,  0.0027811 ,  0.0027811 ,  0.0027811 ,
        0.0027811 ,  0.0027811 ,  0.00279931,  0.00279931]), 9)
(array([ 0.0027811 ,  0.0027811 ,  0.0027811 ,  0.0027811 ,  0.0027811 ,
        0.0027811 ,  0.0027811 ,  0.00279931,  0.00279931,  0.00279931]), 10)
[  1.50000000e+01,  3.69378056e-02,  2.78110126e-03,  2.78110126e-03,
   2.78110126e-03,  2.78110126e-03,  2.78110126e-03,  2.78110126e-03,
   2.78110126e-03,  2.79931203e-03,  2.79931203e-03,  2.79931203e-03]
(array([ 0.00279931]), 1)
(array([ 0.00279931,  0.00279931]), 2)
(array([ 0.00279931,  0.00279931,  0.00279931]), 3)
(array([ 0.00279931,  0.00279931,  0.00279931,  0.00279931]), 4)


KeyboardInterrupt: 